In [1]:
    import pandas as pd
    from sqlalchemy import create_engine
    import psycopg2
    from bs4 import BeautifulSoup
    import requests
    from datetime import datetime,date
    import lxml

In [2]:
police_shooting_df = pd.read_csv('Raw data/police_shooting.csv')
police_shooting_df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2/01/2015,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2/01/2015,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,3/01/2015,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,4/01/2015,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,4/01/2015,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


In [10]:
police_cols = ['id', 'date',"armed", "gender", "race",'city','state']
new_police= police_shooting_df[police_cols].copy()
    
# Clean the data by dropping duplicates and setting the index
# new_police.drop_duplicates("id", inplace=True)
# new_police.set_index("id", inplace=True)
new_police = new_police.drop_duplicates()
# new_police.describe()
new_police

,id,date,armed,gender,race,city,state
0,3,2/01/2015,gun,M,A,Shelton,WA
1,4,2/01/2015,gun,M,W,Aloha,OR
2,5,3/01/2015,unarmed,M,H,Wichita,KS
3,8,4/01/2015,toy weapon,M,W,San Francisco,CA
4,9,4/01/2015,nail gun,M,H,Evans,CO
...,...,...,...,...,...,...,...
5333,5815,20/05/2020,gun,M,NaN,Viera,FL
5334,5836,24/05/2020,gun,M,NaN,Bulls Gap,TN
5335,5837,25/05/2020,knife,M,NaN,Jonesborough,TN
5336,5839,25/05/2020,gun,M,H,Houston,TX


In [13]:
state_name_df = pd.read_csv('Raw data/state_name.csv')
state_name_df.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [65]:
new_police_df = new_police.merge(state_name_df,right_on='Code', left_on = 'state')
new_police_df = new_police_df.drop(['Code'], axis = 1) 
new_police_df = new_police_df.drop(columns=['Abbrev'])
new_police_df

,id,date,armed,gender,race,city,state,State
0,3,2015-01-02,gun,M,A,Shelton,WA,Washington
1,138,2015-02-10,unarmed,M,H,Pasco,WA,Washington
2,295,2015-03-19,unarmed,M,W,Sumas,WA,Washington
3,393,2015-04-21,unarmed,M,N,Lakewood,WA,Washington
4,431,2015-05-04,toy weapon,M,W,Kennewick,WA,Washington
...,...,...,...,...,...,...,...,...
5333,5081,2019-10-01,gun,M,W,Rutland,VT,Vermont
5334,1405,2016-03-24,toy weapon,M,B,Pawtucket,RI,Rhode Island
5335,1621,2016-06-13,gun,M,H,Tiverton,RI,Rhode Island
5336,3119,2017-11-09,NaN,M,W,Providence,RI,Rhode Island


In [ ]:
#string to datetime

In [16]:
new_police['date'] =  pd.to_datetime(new_police['date'],format='%d/%m/%Y')
new_police.dtypes

id                 int64
date      datetime64[ns]
armed             object
gender            object
race              object
city              object
state             object
dtype: object

In [17]:
# gun violence data

In [77]:
gun_violence = pd.read_csv('Raw data/gun_violence.csv')
gun_violence.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,1/01/2013,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,1/01/2013,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,1/01/2013,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,5/01/2013,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,7/01/2013,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [78]:
gun_cols = ['incident_id','date',"state", "city_or_county"]
new_gun= gun_violence[gun_cols].copy()

# Rename the column headers
new_gun = new_gun.rename(columns={"city_or_county": "city",'incident_id':'id','state':'State'})
new_gun

,id,date,State,city
0,461105,1/01/2013,Pennsylvania,Mckeesport
1,460726,1/01/2013,California,Hawthorne
2,478855,1/01/2013,Ohio,Lorain
3,478925,5/01/2013,Colorado,Aurora
4,478959,7/01/2013,North Carolina,Greensboro
...,...,...,...,...
239672,1083142,31/03/2018,Louisiana,Rayne
239673,1083139,31/03/2018,Louisiana,Natchitoches
239674,1083151,31/03/2018,Louisiana,Gretna
239675,1082514,31/03/2018,Texas,Houston


In [79]:
# Clean the data by dropping duplicates and setting the index\n",
new_gun.drop_duplicates("id", inplace=True)
new_gun.set_index("id", inplace=True)
# new_gun = new_gun.duplicated()
# new_gun = new_gun.drop_duplicates("id", inplace=True)
# new_gun.set_index("id", inplace=True)
new_gun

,date,State,city
id,,,
461105,1/01/2013,Pennsylvania,Mckeesport
460726,1/01/2013,California,Hawthorne
478855,1/01/2013,Ohio,Lorain
478925,5/01/2013,Colorado,Aurora
478959,7/01/2013,North Carolina,Greensboro
...,...,...,...
1083142,31/03/2018,Louisiana,Rayne
1083139,31/03/2018,Louisiana,Natchitoches
1083151,31/03/2018,Louisiana,Gretna


In [80]:
#string to datetime\n",
new_gun['date'] =  pd.to_datetime(new_gun['date'],format='%d/%m/%Y')
new_gun.dtypes

date     datetime64[ns]
State            object
city             object
dtype: object

In [81]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_educational_attainment'
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')
edu_table = soup.find_all('table',{'class':'wikitable sortable'})
  
edu_state = pd.read_html(str(edu_table))

edu_df = edu_state[0]
edu_df

,"State, federal district, or territory",% High school graduate or higher,High School rank,% Bachelor's degree or higher,Bachelor's rank,% Advanced degree,Advanced rank
0,Montana,93.0%,1.0,30.7%,21.0,10.1%,33.0
1,New Hampshire,92.8%,2.0,36.0%,9.0,13.8%,10.0
2,Minnesota,92.8%,3.0,34.8%,11.0,11.8%,18.0
3,Wyoming,92.8%,4.0,26.7%,41.0,9.3%,39.0
4,Alaska,92.4%,5.0,29.0%,28.0,10.4%,29.0
5,North Dakota,92.3%,6.0,28.9%,29.0,7.8%,51.0
6,Vermont,92.3%,7.0,36.8%,8.0,15.0%,7.0
7,Maine,92.1%,8.0,30.3%,23.0,10.9%,24.0
8,Iowa,91.8%,9.0,27.7%,36.0,9.0%,42.0
9,Utah,91.8%,10.0,32.5%,16.0,11.0%,23.0


In [112]:
wiki_url_2 = 'https://en.wikipedia.org/wiki/Household_income_in_the_United_States'

response_2 = requests.get(wiki_url_2)
soup_2 = BeautifulSoup(response_2.text, 'html.parser')
income_table = soup_2.find_all('table',{'class':'wikitable sortable'})

income_state = pd.read_html(str(income_table))
income_df = income_state[1]
income_df

,Rank,+/- *,Stateor territory,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,1,+15,District of Columbia,"$82,336","$75,506","$75,628","$71,648","$67,572","$65,246","$66,583","$63,124","$59,290","$57,936","$54,317"
1,2,−1,Maryland,"$80,776","$78,945","$75,847","$73,971","$72,483","$71,836","$70,004","$68,854","$69,272","$70,545","$68,080"
2,3,−1,New Jersey,"$80,088","$76,126","$72,222","$72,919","$70,165","$69,667","$67,458","$67,681","$68,342","$70,373","$67,035"
3,4,+1,Hawaii,"$77,765","$74,511","$73,486","$69,592","$68,020","$66,259","$61,821","$63,030","$64,098","$67,214","$63,746"
4,5,+2,Massachusetts,"$77,385","$75,297","$70,628","$69,160","$66,768","$65,339","$62,859","$62,072","$64,081","$65,401","$62,365"
5,6,−3,Connecticut,"$74,168","$73,433","$71,346","$70,048","$67,098","$67,276","$65,753","$64,032","$67,034","$68,595","$65,967"
6,7,−1,New Hampshire,"$73,381","$70,936","$70,303","$66,532","$64,230","$63,280","$62,647","$61,042","$60,567","$63,731","$62,369"
7,8,−4,Alaska,"$73,181","$76,440","$73,355","$71,583","$72,237","$67,712","$67,825","$64,576","$66,953","$68,460","$64,333"
8,9,−1,California,"$71,805","$67,739","$64,500","$61,933","$60,190","$58,328","$57,287","$57,708","$58,931","$61,021","$59,948"
9,10,−1,Virginia,"$71,535","$68,114","$66,262","$64,902","$62,666","$61,741","$61,882","$60,674","$59,330","$61,233","$59,562"


In [113]:
state_summary = income_df.merge(edu_df,right_on='State, federal district, or territory', left_on = 'Stateor territory')

state_summary_cols = ['Stateor territory', '2017',"2016", "2015", "% High school graduate or higher","% Bachelor's degree or higher",'% Advanced degree']
state_summary= state_summary[state_summary_cols].copy()


# Rename the column headers
state_summary = state_summary.rename(columns={"Stateor territory": "State", "% High school graduate or higher":"High school graduate or higher","% Bachelor's degree or higher":"Bachelor's degree or higher",'% Advanced degree':"Advanced degree"})

state_summary

,State,2017,2016,2015,High school graduate or higher,Bachelor's degree or higher,Advanced degree
0,District of Columbia,"$82,336","$75,506","$75,628",90.3%,56.6%,32.8%
1,Maryland,"$80,776","$78,945","$75,847",89.8%,39.0%,18.0%
2,New Jersey,"$80,088","$76,126","$72,222",89.2%,38.1%,14.7%
3,Hawaii,"$77,765","$74,511","$73,486",91.6%,32.0%,10.8%
4,Massachusetts,"$77,385","$75,297","$70,628",90.3%,42.1%,18.7%
5,Connecticut,"$74,168","$73,433","$71,346",90.2%,38.4%,17.0%
6,New Hampshire,"$73,381","$70,936","$70,303",92.8%,36.0%,13.8%
7,Alaska,"$73,181","$76,440","$73,355",92.4%,29.0%,10.4%
8,California,"$71,805","$67,739","$64,500",82.5%,32.6%,12.2%
9,Virginia,"$71,535","$68,114","$66,262",89.0%,37.6%,16.1%


In [114]:
connection_string = "postgres:sql2007@localhost:5432/shooting_db"
engine = create_engine(f'postgresql://{connection_string}')

In [105]:
new_police.to_sql('police', engine, if_exists='fail');

In [106]:
new_gun.to_sql('gun', engine, if_exists='fail');

In [116]:
state_summary.to_sql('state_summary', engine, if_exists='fail');